# Variables son almacenadas tanto en filas como en columnas

## "Housekeeping"

In [ ]:
#%load_ext lab_black

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option("display.max_columns", 40)

## Ejemplo: Clima

El [Global Historical Climatology Network](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/global-historical-climatology-network-ghcn) recolecta datos de clima de manera diaria. Para este ejemplo, se usan datos de una estación climática en Mexico (MX17004).

### Cargando datos

El raw dataset viene en un formato que es una mezcla de datos de ancho fijo con uso ocasional de caracteres como separadores. Se necesita hacer una limpieza al respecto.

In [6]:
weather_raw = pd.read_csv("data/weather.txt", header=None)
weather_raw
print(weather_raw[21:])

                                                      0
21    MX000017004195511TMAX  270  I  270  I  270  I ...
22    MX000017004195511TMIN  130  I  140  I  130  I ...
23    MX000017004195511PRCP    0  I    0  I    0  I ...
24    MX000017004195512TMAX  240  I  250  I  250  I ...
25    MX000017004195512TMIN  130  I  130  I  140  I ...
...                                                 ...
1709  MX000017004201103TMIN-9999   -9999   -9999   -...
1710  MX000017004201103PRCP    0  S-9999       0  S ...
1711  MX000017004201104TMAX-9999     350  S-9999   -...
1712  MX000017004201104TMIN-9999     168  S-9999   -...
1713  MX000017004201104PRCP    0  S    0  S-9999   -...

[1693 rows x 1 columns]


In [41]:
# Extract the data as one column and
# use string slicing to obtain groups of columns.
weather = pd.read_csv("data/weather.txt", header=None, sep="^")
print(weather)

                                                      0
0     MX000017004195504TMAX  310  I  310  I  310  I ...
1     MX000017004195504TMIN  150  I  150  I  160  I ...
2     MX000017004195504PRCP    0  I    0  I    0  I ...
3     MX000017004195505TMAX  310  I  310  I  310  I ...
4     MX000017004195505TMIN  200  I  160  I  160  I ...
...                                                 ...
1709  MX000017004201103TMIN-9999   -9999   -9999   -...
1710  MX000017004201103PRCP    0  S-9999       0  S ...
1711  MX000017004201104TMAX-9999     350  S-9999   -...
1712  MX000017004201104TMIN-9999     168  S-9999   -...
1713  MX000017004201104PRCP    0  S    0  S-9999   -...

[1714 rows x 1 columns]


In [39]:
weather[weather[0].str.contains("OI")]

0
93   MX000017004195801TMAX  240  I  220  I  210  I ...
466  MX000017004196903TMIN  260 OI  152  I  150  I ...
475  MX000017004196906TMIN  180  I  180  I  210  I ...
508  MX000017004197005TMIN  189  I  193  I  290 OI ...

In [40]:
print(weather[0][93])

MX000017004195801TMAX  240  I  220  I  210  I  210  I  190  I  150  I  220  I  210  I  230  I  260  I  230  I  240  I  250  I  260  I  260  I  240  I  230  I  120  I  110 OI  110 OI  200  I  200  I  170  I  200  I  240  I  250  I  260  I  250  I  250  I  260  I  280  I


In [8]:
# First, remove the weird character separators,
# then split the columns by whitespace, and
# finally name them appropriately.
days = (
    weather[0]
    .map(lambda x: x[21:])
    .str.replace("OI", "  ")
    .str.replace("OS", "  ")
    .str.replace("SI", "  ")
    .str.replace("I", " ")
    .str.replace("S", " ")
    .str.replace("B", " ")
    .str.replace("D", " ")
    .map(str.lstrip)
    .str.split(r"\s+", expand=True)
)[list(range(31))].rename(columns={i: f"d{i+1}" for i in range(31)})
# The non-temperature columns can be extracted as simple slices.
weather = pd.DataFrame(
    data={
        "id": weather[0].map(lambda x: x[:11]),
        "year": weather[0].map(lambda x: x[11:15]).astype(int),
        "month": weather[0].map(lambda x: x[15:17]).astype(int),
        "element": weather[0].map(lambda x: x[17:21]).str.lower(),
    }
)

# The temperatures were stored as whole integers
# with -9999 indicating missing values.
for i in range(1, 32):
    weather[f"d{i}"] = days[f"d{i}"].astype(float) / 10
weather = weather.replace(-999.9, np.NaN)

# Discard the non-temperature observations and
# sort the dataset as in the paper.
weather = (
    weather[weather["element"].isin(["tmax", "tmin"])]
    .sort_values(["id", "year", "month", "element"])
    .reset_index(drop=True)
)

In [9]:
print(weather) #MX000017004 : 21 caracteres. Le sacamos las fechas.

               id  year  month element    d1    d2    d3    d4    d5    d6  \
0     MX000017004  1955      4    tmax  31.0  31.0  31.0  32.0  33.0  32.0   
1     MX000017004  1955      4    tmin  15.0  15.0  16.0  15.0  16.0  16.0   
2     MX000017004  1955      5    tmax  31.0  31.0  31.0  30.0  30.0  30.0   
3     MX000017004  1955      5    tmin  20.0  16.0  16.0  15.0  15.0  15.0   
4     MX000017004  1955      6    tmax  30.0  29.0  28.0  27.0  28.0  26.0   
...           ...   ...    ...     ...   ...   ...   ...   ...   ...   ...   
1124  MX000017004  2011      2    tmin   NaN   NaN   NaN   NaN   NaN   NaN   
1125  MX000017004  2011      3    tmax   NaN   NaN   NaN   NaN  33.2   NaN   
1126  MX000017004  2011      3    tmin   NaN   NaN   NaN   NaN  14.8   NaN   
1127  MX000017004  2011      4    tmax   NaN  35.0   NaN   NaN   NaN   NaN   
1128  MX000017004  2011      4    tmin   NaN  16.8   NaN   NaN   NaN   NaN   

        d7    d8    d9   d10   d11   d12   d13   d14   d15   d1

### Messy Data

A continuación hay un dataset que asumimos que ha sido proporcionado "crudo" como está al analista de datos, es decir, el analista no realizó el trabajo de "parsing" anterior, sino un tercero anteriormente.

> La forma más complicada de *messy data* ocurre cuando las variables se almacenan tanto en filas como en columnas. La Tabla 11 muestra datos meteorológicos diarios del Global Historical Climatology Network para una estación meteorológica (MX17004) en México durante cinco meses en 2010. Tiene variables en
columnas individuales (`"id"`, `"year"`, `"month"`), distribuidas en columnas (`day`, `"d1"`–`"d31"`) y en filas (`"tmin"` y `"tmax"` para las temperaturas mínima y máxima). Los meses con menos de 31 días tienen valores faltantes para los últimos días del mes. La columna `"element"` no es una variable: almacena los *nombres* de las variables.

In [ ]:
weather[(weather["year"] == 2010)].head(10)

### Molten Data

> Para hacer este dataset tidy, primero le hacemos un `melt`  con colvars `"id"`, `"year"`, `"month"`, y la columna que contiene los nombres de las variables reales, `"element"` [...]. Para la presentación, hemos descartado los valores faltantes, haciéndolos implícitos en lugar de explícitos. Esto es permisible porque sabemos cuántos días hay en cada mes y podemos reconstruir fácilmente los valores faltantes explícitos.

In [ ]:
# Melt the dataset and extract a date column.
molten_weather = (
    pd.melt(weather, id_vars=["id", "year", "month", "element"], var_name="day")
    .assign(day=lambda x: x["day"].str.extract("(\d+)").astype(int))
    .assign(date=lambda x: pd.to_datetime(x[["year", "month", "day"]], errors="coerce"))
)
molten_weather = molten_weather[["id", "date", "element", "value"]]

# Make the missing values implicit.
molten_weather = molten_weather[molten_weather["value"].notnull()]

# Sort the data as in the paper.
molten_weather = molten_weather.sort_values(["id", "date", "element"])
molten_weather = molten_weather.reset_index(drop=True)

> Este conjunto de datos está mayormente tidy, pero tenemos dos variables almacenadas en filas: `"tmin"` y `"tmax"`, el tipo de observación.

In [ ]:
molten_weather[(molten_weather["date"].dt.year == 2010)].head(10)

### Tidy Data

> Arreglar esto requiere la operación `cast`(conversión) o `unstack`(desapilar). Esto realiza el inverso del "fundido" (melting), rotando la variable `element` de vuelta hacia las columnas.

A continuación, [pd.DataFrame.unstack()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html) usa un DataFrame's index como columnas para poder desapilar.


In [ ]:
tidy_weather = molten_weather.set_index(["id", "date", "element"]).unstack()

# Make the column headers look as in the paper.
tidy_weather.columns = tidy_weather.columns.droplevel(0)
tidy_weather.columns.name = None
tidy_weather = tidy_weather.reset_index()

> Esta forma está tidy. Hay una variable en cada columna, y cada fila representa la observación de un día.

In [ ]:
tidy_weather[(tidy_weather["date"].dt.year == 2010)].head(10)